In [ ]:
!gdown --id '1lfeLzSgj09icKQT9Yrrvl3quFrwTN9bb' --output data.zip
!unzip data.zip

!gdown --id '1SCU327iM3Yj8aYCpJLpQmocD3G7hGCWf' --output zc.h5

In [ ]:
!pip install git+https://github.com/tensorflow/examples.git

In [ ]:
!rm -rf dataset/error

目标(车牌)检测

In [ ]:
import pathlib

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow_examples.models.pix2pix import pix2pix

width = 416
height = 416

def load_and_preprocess_image(path):
    img = tf.io.read_file(path + '/img.png')
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [width, height])
    img /= 255.0

    label = tf.io.read_file(path + '/label.png')
    label = tf.image.decode_jpeg(label, channels=3)
    label = tf.image.resize(label, [width, height])
    # 3 通道降为 1 通道
    label = tf.image.rgb_to_grayscale(label)
    label /= 38.0

    return img, label


def unet_model(output_channels):
    inputs = tf.keras.layers.Input(shape=[width, height, 3])
    x = inputs

    # 在模型中降频取样
    skips = down_stack(x)
    x = skips[-1]
    skips = reversed(skips[:-1])

    # 升频取样然后建立跳跃连接
    for up, skip in zip(up_stack, skips):
        x = up(x)
        concat = tf.keras.layers.Concatenate()
        x = concat([x, skip])

    # 这是模型的最后一层
    last = tf.keras.layers.Conv2DTranspose(
        output_channels, 3, strides=2,
        padding='same')  # 64x64 -> 128x128

    x = last(x)

    return tf.keras.Model(inputs=inputs, outputs=x)


def display(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    plt.figure(figsize=(16, 16))
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i + 1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()


def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]


def show_predictions(dataset=None, num=2):
    if dataset:
        for img, mask in dataset.take(num):
            pred_mask = model.predict(img)
            display([img[0], mask[0], create_mask(pred_mask)])
    else:
        display([sample_image, sample_mask, create_mask(model.predict(sample_image[tf.newaxis, ...]))])


In [ ]:
batch_size = 32
output_channels = 3
all_image_paths = [str(path) for path in pathlib.Path('./dataset').glob('*/*')]
image_count = len(all_image_paths)
steps_per_epoch = tf.math.ceil(image_count / batch_size).numpy()

In [ ]:
np.random.shuffle(all_image_paths)

valid_count = int((tf.math.floor(image_count / 10)).numpy())

valid_image_path = all_image_paths[0 : valid_count]

train_image_path = all_image_paths[valid_count : ]

In [ ]:
image_ds = (
    tf.data.Dataset.from_tensor_slices(train_image_path)
        .map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
)

train_dataset = (
    image_ds.cache()
        .shuffle(image_count)
        .repeat()
        .batch(batch_size)
        .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
)

for image, mask in image_ds.take(2):
    sample_image, sample_mask = image, mask

display([sample_image, sample_mask])

base_model = tf.keras.applications.MobileNetV2(input_shape=[width, height, 3], include_top=False)

# 使用这些层的激活设置
layer_names = [
    'block_1_expand_relu',  # 64x64
    'block_3_expand_relu',  # 32x32
    'block_6_expand_relu',  # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',  # 4x4
]
layers = [base_model.get_layer(name).output for name in layer_names]

# 创建特征提取模型
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

down_stack.trainable = False

up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),  # 32x32 -> 64x64
]

model = unet_model(output_channels)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
show_predictions()

model_history = model.fit(train_dataset, epochs=10, steps_per_epoch=steps_per_epoch)

show_predictions(train_dataset)

In [ ]:
valid_ds = (
  tf.data.Dataset.from_tensor_slices(valid_image_path)
    .map(load_and_preprocess_image, tf.data.experimental.AUTOTUNE)
    .cache()
    .batch(100)
    .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)    
)

model.evaluate(valid_ds, verbose=2)

2/2 - 7s - loss: 0.0044 - accuracy: 0.9840


[0.004411337897181511, 0.9840132594108582]

In [ ]:
show_predictions(valid_ds, 1)

model.save('zc.h5')

根据人工标记label裁剪车牌

In [ ]:
import pathlib

import cv2
import numpy as np
import tensorflow as tf


def locate(img_src, img_mask, dir_path):
    contours, hierarchy = cv2.findContours(img_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 获取边缘区域宽度最大的区域
    cont = max(contours, key=lambda item: cv2.boundingRect(item)[2])

    x, y, w, h = cv2.boundingRect(cont)  # 获取最小外接矩形

    if w > 15 and h > 15:
        rect = cv2.minAreaRect(cont)  # 针对坐标点获取带方向角的最小外接矩形，中心点坐标，宽高，旋转角度
        box = cv2.boxPoints(rect).astype(np.int32)  # 获取最小外接矩形四个顶点坐标

        cont = cont.reshape(-1, 2).tolist()
        # 由于转换矩阵的两组坐标位置需要一一对应，因此需要将最小外接矩形的坐标进行排序，最终排序为[左上，左下，右上，右下]
        box = sorted(box, key=lambda xy: xy[0])  # 先按照左右进行排序，分为左侧的坐标和右侧的坐标
        box_left, box_right = box[:2], box[2:]  # 此时box的前2个是左侧的坐标，后2个是右侧的坐标
        box_left = sorted(box_left, key=lambda x: x[1])  # 再按照上下即y进行排序，此时box_left中为左上和左下两个端点坐标
        box_right = sorted(box_right, key=lambda x: x[1])  # 此时box_right中为右上和右下两个端点坐标
        box = np.array(box_left + box_right)  # [左上，左下，右上，右下]

        x0, y0 = box[0][0], box[0][1]  # 这里的4个坐标即为最小外接矩形的四个坐标，接下来需获取平行(或不规则)四边形的坐标
        x1, y1 = box[1][0], box[1][1]
        x2, y2 = box[2][0], box[2][1]
        x3, y3 = box[3][0], box[3][1]

        def point_to_line_distance(X, Y):
            if x2 - x0:
                k_up = (y2 - y0) / (x2 - x0)  # 斜率不为无穷大
                d_up = abs(k_up * X - Y + y2 - k_up * x2) / (k_up ** 2 + 1) ** 0.5
            else:  # 斜率无穷大
                d_up = abs(X - x2)
            if x1 - x3:
                k_down = (y1 - y3) / (x1 - x3)  # 斜率不为无穷大
                d_down = abs(k_down * X - Y + y1 - k_down * x1) / (k_down ** 2 + 1) ** 0.5
            else:  # 斜率无穷大
                d_down = abs(X - x1)
            return d_up, d_down

        d0, d1, d2, d3 = np.inf, np.inf, np.inf, np.inf
        l0, l1, l2, l3 = (x0, y0), (x1, y1), (x2, y2), (x3, y3)
        for each in cont:  # 计算cont中的坐标与矩形四个坐标的距离以及到上下两条直线的距离，对距离和进行权重的添加，成功选出四边形的4个顶点坐标
            x, y = each[0], each[1]
            dis0 = (x - x0) ** 2 + (y - y0) ** 2
            dis1 = (x - x1) ** 2 + (y - y1) ** 2
            dis2 = (x - x2) ** 2 + (y - y2) ** 2
            dis3 = (x - x3) ** 2 + (y - y3) ** 2
            d_up, d_down = point_to_line_distance(x, y)
            weight = 0.975
            if weight * d_up + (1 - weight) * dis0 < d0:
                d0 = weight * d_up + (1 - weight) * dis0
                l0 = (x - 4, y - 2)
            if weight * d_down + (1 - weight) * dis1 < d1:
                d1 = weight * d_down + (1 - weight) * dis1
                l1 = (x - 4, y + 2)
            if weight * d_up + (1 - weight) * dis2 < d2:
                d2 = weight * d_up + (1 - weight) * dis2
                l2 = (x + 4, y - 2)
            if weight * d_down + (1 - weight) * dis3 < d3:
                d3 = weight * d_down + (1 - weight) * dis3
                l3 = (x + 4, y + 2)

        p0 = np.float32([l0, l1, l2, l3])  # 左上角，左下角，右上角，右下角，形成的新box顺序需和原box中的顺序对应，以进行转换矩阵的形成
        p1 = np.float32([(0, 0), (0, 80), (240, 0), (240, 80)])
        transform_mat = cv2.getPerspectiveTransform(p0, p1)  # 构成转换矩阵
        lic = cv2.warpPerspective(img_src, transform_mat, (240, 80))  # 进行车牌矫正

        print('saving ' + dir_path)
        tf.io.write_file(dir_path + '/plate.jpeg', tf.image.encode_jpeg(lic))


def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]


image_dirs = [str(path) for path in pathlib.Path('dataset').glob('*/*')]

for dir_path in image_dirs:
    file_path = dir_path + '/img.png'
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)

    mask = tf.io.read_file(dir_path + '/label.png')
    mask = tf.image.decode_jpeg(mask, channels=1)

    image = np.asarray(image)
    mask = np.asarray(mask)

    locate(image, mask, str(pathlib.Path(file_path).parent))

print('finish')


利用裁剪出来的车牌训练内容识别

In [ ]:
import pathlib

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

char_dict = {"京": 0, "沪": 1, "津": 2, "渝": 3, "冀": 4, "晋": 5, "蒙": 6, "辽": 7, "吉": 8, "黑": 9, "苏": 10,
             "浙": 11, "皖": 12, "闽": 13, "赣": 14, "鲁": 15, "豫": 16, "鄂": 17, "湘": 18, "粤": 19, "桂": 20,
             "琼": 21, "川": 22, "贵": 23, "云": 24, "藏": 25, "陕": 26, "甘": 27, "青": 28, "宁": 29, "新": 30,
             "0": 31, "1": 32, "2": 33, "3": 34, "4": 35, "5": 36, "6": 37, "7": 38, "8": 39, "9": 40,
             "A": 41, "B": 42, "C": 43, "D": 44, "E": 45, "F": 46, "G": 47, "H": 48, "J": 49, "K": 50,
             "L": 51, "M": 52, "N": 53, "P": 54, "Q": 55, "R": 56, "S": 57, "T": 58, "U": 59, "V": 60,
             "W": 61, "X": 62, "Y": 63, "Z": 64, "": 65}

# 读取数据集
all_image_path = [str(p) for p in pathlib.Path('./dataset').glob('*/*')]

n = len(all_image_path)
X_train, y_train = [], []
idx = 0
for i in range(n):
    path = all_image_path[i]
    print('正在读取 {}'.format(all_image_path[i]))
    img = tf.io.read_file(path + '/plate.jpeg')
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [80, 240])
    img /= 255.0

    plate = pathlib.Path(path).name
    label = [char_dict[name] for name in plate[0:8]]  # 图片名前7位为车牌标签
    if len(label) == 7:
      label.append(65)
    X_train.append(img)
    y_train.append(label)
    if idx == 0:
      plt.imshow(img)
      plt.show()
      idx = 1

X_train = np.array(X_train)
y_train = [np.array(y_train)[:, i] for i in range(8)]

input = tf.keras.layers.Input((80, 240, 3))
x = input
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), strides=1, padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same', strides=2)(x)
for i in range(3):
    x = tf.keras.layers.Conv2D(filters=32 * 2 ** i, kernel_size=(3, 3), padding='valid', activation='relu')(x)
    x = tf.keras.layers.Conv2D(filters=32 * 2 ** i, kernel_size=(3, 3), padding='valid', activation='relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same', strides=2)(x)
    x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.2)(x)
Output = [tf.keras.layers.Dense(66, activation='softmax', name='c%d' % (i + 1))(x) for i in range(8)]

model = tf.keras.models.Model(inputs=input, outputs=Output)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train, y_train, epochs=50)

In [ ]:
model.evaluate(X_train, y_train)

model.save('plate.h5')

定位车牌和车牌识别


In [ ]:
import pathlib

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import cv2


char_dict = {"京": 0, "沪": 1, "津": 2, "渝": 3, "冀": 4, "晋": 5, "蒙": 6, "辽": 7, "吉": 8, "黑": 9, "苏": 10,
             "浙": 11, "皖": 12, "闽": 13, "赣": 14, "鲁": 15, "豫": 16, "鄂": 17, "湘": 18, "粤": 19, "桂": 20,
             "琼": 21, "川": 22, "贵": 23, "云": 24, "藏": 25, "陕": 26, "甘": 27, "青": 28, "宁": 29, "新": 30,
             "0": 31, "1": 32, "2": 33, "3": 34, "4": 35, "5": 36, "6": 37, "7": 38, "8": 39, "9": 40,
             "A": 41, "B": 42, "C": 43, "D": 44, "E": 45, "F": 46, "G": 47, "H": 48, "J": 49, "K": 50,
             "L": 51, "M": 52, "N": 53, "P": 54, "Q": 55, "R": 56, "S": 57, "T": 58, "U": 59, "V": 60,
             "W": 61, "X": 62, "Y": 63, "Z": 64, "": 65}

index_to_char = [
    "京", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏",
    "浙", "皖", "闽", "赣", "鲁", "豫", "鄂", "湘", "粤", "桂", "琼",
    "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新",
    "0", "1", "2", "3", "4", "5", "6", "7", "8", "9",
    "A", "B", "C", "D", "E", "F", "G", "H", "J", "K",
    "L", "M", "N", "P", "Q", "R", "S", "T", "U", "V",
    "W", "X", "Y", "Z", ""
]


def locate(img_src, img_mask):
    contours, hierarchy = cv2.findContours(img_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 获取边缘区域宽度最大的区域
    cont = max(contours, key=lambda item: cv2.boundingRect(item)[2])

    x, y, w, h = cv2.boundingRect(cont)  # 获取最小外接矩形

    if w > 15 and h > 15:
        rect = cv2.minAreaRect(cont)  # 针对坐标点获取带方向角的最小外接矩形，中心点坐标，宽高，旋转角度
        box = cv2.boxPoints(rect).astype(np.int32)  # 获取最小外接矩形四个顶点坐标

        cont = cont.reshape(-1, 2).tolist()
        # 由于转换矩阵的两组坐标位置需要一一对应，因此需要将最小外接矩形的坐标进行排序，最终排序为[左上，左下，右上，右下]
        box = sorted(box, key=lambda xy: xy[0])  # 先按照左右进行排序，分为左侧的坐标和右侧的坐标
        box_left, box_right = box[:2], box[2:]  # 此时box的前2个是左侧的坐标，后2个是右侧的坐标
        box_left = sorted(box_left, key=lambda x: x[1])  # 再按照上下即y进行排序，此时box_left中为左上和左下两个端点坐标
        box_right = sorted(box_right, key=lambda x: x[1])  # 此时box_right中为右上和右下两个端点坐标
        box = np.array(box_left + box_right)  # [左上，左下，右上，右下]

        x0, y0 = box[0][0], box[0][1]  # 这里的4个坐标即为最小外接矩形的四个坐标，接下来需获取平行(或不规则)四边形的坐标
        x1, y1 = box[1][0], box[1][1]
        x2, y2 = box[2][0], box[2][1]
        x3, y3 = box[3][0], box[3][1]

        def point_to_line_distance(X, Y):
            if x2 - x0:
                k_up = (y2 - y0) / (x2 - x0)  # 斜率不为无穷大
                d_up = abs(k_up * X - Y + y2 - k_up * x2) / (k_up ** 2 + 1) ** 0.5
            else:  # 斜率无穷大
                d_up = abs(X - x2)
            if x1 - x3:
                k_down = (y1 - y3) / (x1 - x3)  # 斜率不为无穷大
                d_down = abs(k_down * X - Y + y1 - k_down * x1) / (k_down ** 2 + 1) ** 0.5
            else:  # 斜率无穷大
                d_down = abs(X - x1)
            return d_up, d_down

        d0, d1, d2, d3 = np.inf, np.inf, np.inf, np.inf
        l0, l1, l2, l3 = (x0, y0), (x1, y1), (x2, y2), (x3, y3)
        for each in cont:  # 计算cont中的坐标与矩形四个坐标的距离以及到上下两条直线的距离，对距离和进行权重的添加，成功选出四边形的4个顶点坐标
            x, y = each[0], each[1]
            dis0 = (x - x0) ** 2 + (y - y0) ** 2
            dis1 = (x - x1) ** 2 + (y - y1) ** 2
            dis2 = (x - x2) ** 2 + (y - y2) ** 2
            dis3 = (x - x3) ** 2 + (y - y3) ** 2
            d_up, d_down = point_to_line_distance(x, y)
            weight = 0.975
            if weight * d_up + (1 - weight) * dis0 < d0:
                d0 = weight * d_up + (1 - weight) * dis0
                l0 = (x - 4, y - 2)
            if weight * d_down + (1 - weight) * dis1 < d1:
                d1 = weight * d_down + (1 - weight) * dis1
                l1 = (x - 4, y + 2)
            if weight * d_up + (1 - weight) * dis2 < d2:
                d2 = weight * d_up + (1 - weight) * dis2
                l2 = (x + 4, y - 2)
            if weight * d_down + (1 - weight) * dis3 < d3:
                d3 = weight * d_down + (1 - weight) * dis3
                l3 = (x + 4, y + 2)

        p0 = np.float32([l0, l1, l2, l3])  # 左上角，左下角，右上角，右下角，形成的新box顺序需和原box中的顺序对应，以进行转换矩阵的形成
        p1 = np.float32([(0, 0), (0, 80), (240, 0), (240, 80)])
        transform_mat = cv2.getPerspectiveTransform(p0, p1)  # 构成转换矩阵
        lic = cv2.warpPerspective(img_src, transform_mat, (240, 80))  # 进行车牌矫正

        return lic


width = 416
height = 416
images_path = [str(p) for p in pathlib.Path('./dataset').glob('*/*')]
detect_model = tf.keras.models.load_model('zc.h5')
recognition_model = tf.keras.models.load_model('plate.h5')

np.random.shuffle(images_path)

error_count = 0
idx = 0
for p in images_path:
    print('idx {}'.format(idx))
    idx += 1
    img = tf.io.read_file(p + '/img.png')
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [width, height])
    raw_img = img
    img = img / 255.0


    result = detect_model.predict(np.array([img]))

    mask = create_mask(result)
    mask = tf.keras.preprocessing.image.array_to_img(mask)
    mask = np.asarray(mask)
    img = np.asarray(img)

    plate_image = locate(img, mask)
    plate_chars = recognition_model.predict(np.array([plate_image]))
    plate = []
    for cs in plate_chars:
        plate.append(index_to_char[np.argmax(cs)])

    real_plate = pathlib.Path(p).name
    predict_plate = ''.join(plate)

    if predict_plate != real_plate:
        print('wrong real plate is {}, predict plate is {}'.format(real_plate, predict_plate))
        error_count += 1
        raw_img = np.asarray(raw_img)
        plate_image = locate(raw_img, mask)
        tf.io.write_file('./dataset/error/' + real_plate + '/plate.jpeg', tf.image.encode_jpeg(plate_image))

print('total error {}'.format(error_count))